In [5]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sopt
from pysimu import ode2numba, ssa

from ipywidgets import *


In [6]:
%matplotlib notebook

## System definition 

In [10]:
S_base = 100.0e6
U_base = 20e3
Z_base = U_base**2/S_base
r_km = 0.127 # ohm/km
x_km = 0.113 # ohm/km
lenght = 1.0
R = r_km*lenght/Z_base
X = x_km*lenght/Z_base
Z = R +1j*X
Y = 1.0/Z
G_s_inf, B_s_inf = Y.real, Y.imag

sys = { 't_end':20.0,'Dt':0.01,'solver':'forward-euler', 'decimation':10, 'name':'vsg_pi_i',
   'models':[{'params':
                   {'K_p':0.01,
                    'K_i':0.1,
                    'K_q':0.1,
                    'T_q':0.1,
                    'Omega_b' : 2*np.pi*50,
                    'R_g':0.01,
                    'X_g':0.05,
                    'V_g': 1.0,
                    'K_f': 0.0,
                    'K_s': 1.0,
                    'H': 5.0,
                    'R_s':0.01,
                    'X_s':0.01,
                   },
              'f':[
                    'dphi_s = Omega_b*(omega_s - 1)',
                    'dxi_p  = epsilon_p',
                    'dxi_q  = epsilon_q',
                    'dphi_g = Omega_b*(omega_g - 1) -1e-5*phi_g',
                    'domega_g = RoCoFpu - 1e-5*(omega_g - 1)'
                  ],   
              'g':[
                   'omega_s @ -omega_s + K_p*epsilon_p + K_i*xi_p + 1',
                   'p_s_ref @ -p_s_ref + p_m',
                   'p_h  @ p_h + 2*H*RoCoFpu',
                   'p_m0 @ p_m0 - p_m',
                   'v_gr @-v_gr + V_g*cos(phi_g)',
                   'v_gi @-v_gi + V_g*sin(phi_g)',
                   'v_sr @ -v_sr + v_gr + R_g*i_sr - X_g*i_si', # kron reference frame
                   'v_si @ -v_si + v_gi + R_g*i_si + X_g*i_sr', # kron reference frame
                   'v_sd_s@-v_sd_s + v_si', # phasor to dq in POI reference frame (v_sd_s = 0)
                   'v_sq_s@-v_sq_s - v_sr', # phasor to dq                   
                   'v_sd @-v_sd + cos(phi_s)*v_sd_s + cos(phi_s-pi/2)*v_sq_s',# virtual rotor reference frame
                   'v_sq @-v_sq - sin(phi_s)*v_sd_s - sin(phi_s-pi/2)*v_sq_s',# virtual rotor reference frame
                   'i_sd @ -v_sd    + 0.0                        + R_s*i_sd + X_s*i_sq',# virtual rotor reference frame
                   'i_sq @ -v_sq -1 + K_q*(epsilon_q + xi_q/T_q) + R_s*i_sq - X_s*i_sd',# virtual rotor reference frame
                   'i_sd_s @ -i_sd_s + cos(phi_s)*i_sd      - sin(phi_s)*i_sq', # kron reference frame
                   'i_sq_s @ -i_sq_s + cos(phi_s-pi/2)*i_sd - sin(phi_s-pi/2)*i_sq', # kron reference frame
                   'i_si@-i_si + i_sd_s', # dq to phasor  
                   'i_sr@-i_sr - i_sq_s', # dq to phasor     
                   'epsilon_p@-epsilon_p + p_s_ref - p_s',
                   'epsilon_q@-epsilon_q + q_s_ref - q_s',
                   'p_s@-p_s+ i_sd*v_sd + i_sq*v_sq', # active power equation 
                   'q_s@-q_s+ i_sd*v_sq - i_sq*v_sd', # reactive power equation 
                  ],
              'u':{'p_m':0.8,'q_s_ref':0.1, 'RoCoFpu':0.0},
              'y':['omega_s','p_s_ref','p_h','p_m0','v_gr','v_gi','i_sr', 'i_si', 'i_sd_s', 'i_sq_s', 'i_sd','i_sq','v_sd','v_sq','v_sd_s','v_sq_s',
                   'v_sr','v_si','epsilon_p','epsilon_q','p_s','q_s'],
             'y_ini':['omega_s','p_s_ref','p_h','p_m0','v_gr','v_gi','i_sr', 'i_si', 'i_sd_s', 'i_sq_s', 'i_sd','i_sq','v_sd','v_sq','v_sd_s','v_sq_s',
                   'v_sr','v_si','epsilon_p','epsilon_q','p_s','q_s'],

              'h':[
                   'p_m'
                   ]}
              ],
    'perturbations':[{'type':'step','time':100.0,'var':'p_m','final':1.01} ]
    }

x,f = ode2numba.system(sys)  ;


omega_s 
p_s_ref 
p_h  
p_m0 
v_gr 
v_gi 
v_sr 
v_si 
v_sd_s
v_sq_s
v_sd 
v_sq 
i_sd 
i_sq 
i_sd_s 
i_sq_s 
i_si
i_sr
epsilon_p
epsilon_q
p_s
q_s
 -omega_s + K_p*epsilon_p + K_i*xi_p + 1,  -p_s_ref + p_m,  p_h + 2*H*RoCoFpu,  p_m0 - p_m, -v_gr + V_g*cos(phi_g), -v_gi + V_g*sin(phi_g),  -v_sr + v_gr + R_g*i_sr - X_g*i_si,  -v_si + v_gi + R_g*i_si + X_g*i_sr, -v_sd_s + v_si, -v_sq_s - v_sr, -v_sd + cos(phi_s)*v_sd_s + cos(phi_s-pi/2)*v_sq_s, -v_sq - sin(phi_s)*v_sd_s - sin(phi_s-pi/2)*v_sq_s,  -v_sd    + 0.0                        + R_s*i_sd + X_s*i_sq,  -v_sq -1 + K_q*(epsilon_q + xi_q/T_q) + R_s*i_sq - X_s*i_sd,  -i_sd_s + cos(phi_s)*i_sd      - sin(phi_s)*i_sq,  -i_sq_s + cos(phi_s-pi/2)*i_sd - sin(phi_s-pi/2)*i_sq, -i_si + i_sd_s, -i_sr - i_sq_s, -epsilon_p + p_s_ref - p_s, -epsilon_q + q_s_ref - q_s, -p_s+ i_sd*v_sd + i_sq*v_sq, -q_s+ i_sd*v_sq - i_sq*v_sd, 
Matrix([[omega_s], [p_s_ref], [p_h], [p_m0], [v_gr], [v_gi], [v_sr], [v_si], [v_sd_s], [v_sq_s], [v_sd], [v_sq], [i_sd], [i_sq

In [35]:
25/40*25000

15625.0

In [36]:
15625.0/12

1302.0833333333333